In [ ]:
#importing libraries
!pip install sentence-transformers
!pip install translators
!pip install flask-ngrok
!pip install flasgger

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
import spacy
nlp = spacy.load('en_core_web_sm')
import translators as ts
import pandas as pd
import numpy as np
import scipy
from scipy import spatial
from scipy.spatial import distance

from flask import Flask, request
import flasgger
from flasgger import Swagger
from flask_ngrok import run_with_ngrok

In [ ]:
#Utility functions
def augment(text,lang):
  text1 = ts.google(text,from_language = 'en', to_language=lang)
  text2 = ts.google(text1,from_language = lang, to_language='en')
  return text2

#Parameters
lang_codes = ['ar','ru','hi','fr','de','it','ja','ko','tr','th']
threshold_score = 0.95

##Main process function
def process(text):
  emb1 = model.encode(text, show_progress_bar=False)
  ans = []
  ans_sent = []
  for each in lang_codes:
    aug_text = augment(text,each)

    #Duplicate checking
    if each == aug_text or aug_text in ans_sent:
      continue
    else:
      emb2 = model.encode(aug_text, show_progress_bar=False) 
      c_s = 1 - spatial.distance.cosine(emb1, emb2)

      #Threshold comparison
      if c_s > threshold_score and c_s != 1:
        #Printing the results
        print(aug_text)
        ans_sent.append(aug_text)
        ans.append([c_s,aug_text])
  
  #If no augmentation works
  if len(ans)==0:
    return ["Sorry no augmented sentences could be generated for the input sentence."] 
  
  ans = sorted(ans,reverse = True)
  return ans

In [ ]:
#Testing the Pipeline
text = "In the end, we all felt like we ate too much."
a = process(text)
print('\n'.join(map(str, a)))

In the end, we all felt that we ate too much.
Finally, we all felt like we ate too much.
In the end, we all felt like we eat too much.
In the end we all felt, as if we had eaten too much.
Finally, we felt that we all have been eating too much.
In the end, we all feel like we eat too much.
[0.9937810897827148, 'In the end, we all felt like we eat too much.']
[0.9931422472000122, 'In the end, we all felt that we ate too much.']
[0.9800915718078613, 'In the end we all felt, as if we had eaten too much.']
[0.9793198704719543, 'In the end, we all feel like we eat too much.']
[0.9684463739395142, 'Finally, we felt that we all have been eating too much.']
[0.9646124839782715, 'Finally, we all felt like we ate too much.']


## Flask App


In [ ]:
app = Flask(__name__)
run_with_ngrok(app) 
Swagger(app)


@app.route('/')
def welcome():
    msg = "Welcome Everyone. Please add '/apidocs' at the end of the URL for a proper UI."
    return msg


@app.route('/augment', methods=["Get"]) 
def augment_the_sentence():
    """ Let's augment some sentences
    ---
    tags:
      - Automated Sentence Augmenter
    parameters:  
      - name: text
        in: query
        type: string
        required: true
        description: Type a sentence here
    responses:
        200:
          description: A List of potential augmented sentences 
        500:
          description: The input sentence was not processed properly
     
    """
    input_sentence = request.args.get("text")
    print(input_sentence)
    prediction = process(str(input_sentence))
    x = '\n'.join(map(str, prediction))
    print(x)
    return x

In [ ]:
if __name__  == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://928f-35-185-34-187.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Aug/2021 10:23:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2021 10:23:45] "GET /apidocs HTTP/1.1" 308 -
127.0.0.1 - - [26/Aug/2021 10:23:46] "GET /apidocs/ HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:46] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:46] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:46] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:46] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:47] "GET /flasgger_static/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2021 10:23:47] "GET /apispec_1.json HTTP/1.1" 200 -


In the end, we all felt like we ate too much.
In the end, we all felt that we ate too much.
Finally, we all felt like we ate too much.
In the end, we all felt like we eat too much.
In the end we all felt, as if we had eaten too much.
Finally, we felt that we all have been eating too much.


127.0.0.1 - - [26/Aug/2021 10:24:10] "GET /augment?text=In%20the%20end%2C%20we%20all%20felt%20like%20we%20ate%20too%20much. HTTP/1.1" 200 -


In the end, we all feel like we eat too much.
[0.9937810897827148, 'In the end, we all felt like we eat too much.']
[0.9931422472000122, 'In the end, we all felt that we ate too much.']
[0.9800915718078613, 'In the end we all felt, as if we had eaten too much.']
[0.9793198704719543, 'In the end, we all feel like we eat too much.']
[0.9684463739395142, 'Finally, we felt that we all have been eating too much.']
[0.9646124839782715, 'Finally, we all felt like we ate too much.']
